# Loan Data Analysis Project
### Elaina S, 12-29-2021

## Import Numpy Package

In [1]:
import numpy as np

Supress = True stops NumPy from using scientific notation for numbers.
linewidth = 100 extends the number of characters in a line of output to 100.
precision = 2 will only display 2 digits after the decimal point.
These settings help prevent rows of the array being displayed over multiple lines, for easier viewing of results.

In [2]:
np.set_printoptions(suppress = True, linewidth = 100, precision = 2)

## Import the Data from the .csv File

In [3]:
raw_data_np = np.genfromtxt("loan-data.csv", delimiter = ';', encoding = 'ISO8859', skip_header = 1, autostrip = True)
raw_data_np

array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

raw_data_np has nan values, which could be text or missing elements.
skip_header omits the first line, which is the names of the columns. 
auostrip True removes excess white spaces since they can distort the columns.

## Checking for Incomplete Data

There are 88005 nan (not a number) values in the data. 

In [4]:
np.isnan(raw_data_np).sum()

88005

In [5]:
temporary_fill = np.nanmax(raw_data_np) + 1
temporary_mean = np.nanmean(raw_data_np, axis = 0)

/Users/elainasmith/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: Mean of empty slice
  


nanmax() and nanmean return the max and mean of an array, ignoring nan values.
temporary_fill is a temporary filler for all missing entries in the dataset.
temporary_mean holds the means for every column.

The call to nanmean creates 'Runtime Warning: Mean of empty slice.' Meaning there are columns in the data that are all nan or string values. The default data type for genfromtxt() is float, so a column of only strings is automatically filled with nan values and considered empty. So the data may contain important text data. Looking at the .csv data, we know that the data has string values. While errors prevent the program from compiling, warnings do not, so we can proceed.  

In [6]:
temporary_mean

array([54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
            440.92,         nan,         nan,         nan,         nan,         nan,     3143.85])

In [7]:
np.isnan(temporary_mean).sum()

8

temporary_mean shows that 8 columns have nan as their mean, meaning those columns have no floats.  
Storing numbers and text in the same array limits what we can do with the data set. Splitting the data into a number array and a string array will solve this. First, get the minimum and maximum values of each numeric column using nanmin and nanmax. We know the warnings are due to some columns not containing floats.

In [8]:
temporary_stats = np.array(
    [np.nanmin(raw_data_np, axis = 0), 
     temporary_mean, 
     np.nanmax(raw_data_np, axis = 0)])

/Users/elainasmith/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: All-NaN slice encountered
  
/Users/elainasmith/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: All-NaN slice encountered
  after removing the cwd from sys.path.


In [9]:
temporary_stats

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

temporary_stats is a 2-D array that holds 3 1-dimensional arrays. The first array is the min of each column, the second array is the mean of each column, and the third array is the max of each column. 

## Splitting the Dataset

In [10]:
columns_strings = np.argwhere(np.isnan(temporary_mean)).squeeze()
columns_strings

array([ 1,  3,  5,  8,  9, 10, 11, 12])

In [11]:
columns_numeric = np.argwhere(np.isnan(temporary_mean) == False).squeeze()
columns_numeric

array([ 0,  2,  4,  6,  7, 13])

If a column contains only text, its mean is nan. Then .isnan() will return true for the mean of that column. The argwhere() method will return the indices of an array where a given expression does not evaluate to 0. Without .squeeze(), each value is in a separate vector and the array is a 2D array. .sqeeze() stores the results in a 1-dimensional array. 

columns_strings is an array containing the indices of the string columns and columns_numeric is an array containing the indices of the numeric columns.

## Re-Import the Dataset as 2 Separate Arrays

In [12]:
loan_data_strings = np.genfromtxt("loan-data.csv", dtype = np.str, delimiter = ';', skip_header = 1, usecols = columns_strings, autostrip = True, encoding = 'ISO8859')

loan_data_strings

array([['May-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=48010226', 'CA'],
       ['', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=57693261', 'NY'],
       ['Sep-15', 'Current', '36 months', ..., 'Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=59432726', 'PA'],
       ...,
       ['Jun-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=50415990', 'CA'],
       ['Apr-15', 'Current', '36 months', ..., 'Source Verified',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=46154151', 'OH'],
       ['Dec-15', 'Current', '36 months', ..., '',
        'https://www.lendingclub.com/browse/loanDetail.action?loan_id=66055249', 'IL']],
      dtype='<U69')

In [13]:
loan_data_numeric = np.genfromtxt("loan-data.csv", dtype = np.str, delimiter = ';', skip_header = 1, filling_values = temporary_fill, usecols = columns_numeric, autostrip = True, encoding = 'ISO8859')

loan_data_numeric

array([['48010226', '35000.0', '35000.0', '13.33', '1184.86', '9452.96'],
       ['57693261', '30000.0', '30000.0', 'þëè.89', '938.57', '4679.7'],
       ['59432726', '15000.0', '15000.0', 'íîå.53', '494.86', '1969.83'],
       ...,
       ['50415990', '10000.0', '10000.0', 'þëè.89', '', '2185.64'],
       ['46154151', '', '10000.0', '16.55', '354.3', '3199.4'],
       ['66055249', '10000.0', '10000.0', 'þëè.26', '309.97', '301.9']], dtype='<U11')

We now use the optional data type parameter, dtype, to specify the data type of the resulting array from calling genfromtxt(). Therefore both loan_data_strings and loan_data_numeric contain the string data type. loan_data_strings is the string dataset and loan_data_numeric is the numeric dataset. loan_data_strings has only the string values of each row and loan_data_numeric only has the numeric values of each row because we used the usecols parameter. We will be manipulating these datasets separately due to handling missing values differently for the strings and numbers. 

In [14]:
header_full = np.genfromtxt("loan-data.csv", dtype = np.str, delimiter = ';', skip_footer = raw_data_np.shape[0], autostrip = True, encoding = 'ISO8859')
header_full


array(['id', 'issue_d', 'loan_amnt', 'loan_status', 'funded_amnt', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'verification_status', 'url', 'addr_state',
       'total_pymnt'], dtype='<U19')

We set skip_footer to the number of rows in raw_data_np because we are telling .genfromtxt to exclude all rows after the header, which is the first row at index 0. header_full contains all the column names.

In [15]:
header_strings, header_numeric = header_full[columns_strings], header_full[columns_numeric]

header_strings is an array containing the string column names, and header_numeric is an array containing the numeric column names.

In [16]:
header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

In [17]:
header_numeric

array(['id', 'loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'total_pymnt'], dtype='<U19')

## Create a checkpoint to store a copy of the dataset and avoid losing progress

In [18]:
def checkpoint(file_name, checkpoint_header, checkpoint_data):
    np.savez(file_name, header = checkpoint_header, data = checkpoint_data) #Create .npz file.
    checkpoint_variable = np.load(file_name + ".npz") #Load the file we just saved into a checkpoint variable.
    return(checkpoint_variable) #Return the checkpoint.

In [19]:
checkpoint1 = checkpoint('20211229_backup_string_loan_data', header_strings, loan_data_strings)

In [20]:
np.array_equal(checkpoint1['data'], loan_data_strings)

True

In [21]:
checkpoint2 = checkpoint('20211229_backup_numeric_loan_data', header_numeric, loan_data_numeric)

In [22]:
np.array_equal(checkpoint2['data'], loan_data_numeric)

True

## Manipulate String Columns

### Issue Date

In [23]:
header_strings

array(['issue_d', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

Give the columns descriptive names.

In [24]:
header_strings[0] = 'issue_date'

In [25]:
header_strings

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

Take a slice of the multidimensional array, keeping all rows but only selecting the first column. 

In [26]:
loan_data_strings[:,0]

array(['May-15', '', 'Sep-15', ..., 'Jun-15', 'Apr-15', 'Dec-15'], dtype='<U69')

Use the unique() method to see all the unique values in the array.

In [27]:
np.unique(loan_data_strings[:,0])

array(['', 'Apr-15', 'Aug-15', 'Dec-15', 'Feb-15', 'Jan-15', 'Jul-15', 'Jun-15', 'Mar-15',
       'May-15', 'Nov-15', 'Oct-15', 'Sep-15'], dtype='<U69')

Each of the column names is the fifteenth of the month, and so '-15' can be removed because that is not a meaningful differentiator in the data. 

In [28]:
loan_data_strings[:,0] = np.chararray.strip(loan_data_strings[:,0], "-15")

In [29]:
np.unique(loan_data_strings[:,0])

array(['', 'Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct', 'Sep'],
      dtype='<U69')

The month values can be represented as integers, e.g. 1 for January, because they use less memory compared to using the names of the months. 

Missing values will be represented with 0. 0 will be used for candidates with no loan issue date.

In [30]:
months = np.array(['0', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

In [31]:
#range() uses a closed-open interval and so the loop variable, i, will go from 0 up to but not including 13. 
for i in range(13):
    loan_data_strings[:,0] = np.where(loan_data_strings[:,0] == months[i], i, loan_data_strings[:,0])

In [32]:
np.unique(loan_data_strings[:,0])

array(['', '1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'], dtype='<U69')

### Loan Status

In [33]:
header_strings

array(['issue_date', 'loan_status', 'term', 'grade', 'sub_grade', 'verification_status', 'url',
       'addr_state'], dtype='<U19')

The Loan Status column is at index 1 in the header_strings array, the second column.

In [36]:
np.unique(loan_data_strings[:,1])

array(['', 'Charged Off', 'Current', 'Default', 'Fully Paid', 'In Grace Period', 'Issued',
       'Late (16-30 days)', 'Late (31-120 days)'], dtype='<U69')

In [35]:
np.unique(loan_data_strings[:,1]).size

9

The 9 loan statuses can be simplified into 2 categories: 1, which represents good, non-defaulted accounts; and 0, which represents bad, defaulted accounts.

In [39]:
bad_status = np.array(['', 'Charged Off', 'Default', 'Late (31-120 days)'])

In [40]:
loan_data_strings[:,1] = np.where(np.isin(loan_data_strings[:,1], bad_status), 0, 1)

In [41]:
np.unique(loan_data_strings[:,1])

array(['0', '1'], dtype='<U69')